In [1]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score, train_test_split, KFold, cross_val_predict
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
%matplotlib inline
from sklearn.linear_model import LinearRegression, RidgeCV, Lasso, ElasticNetCV, BayesianRidge, LassoLarsIC
from sklearn.metrics import mean_squared_error, make_scorer 
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
import math
import matplotlib.pyplot as plt
import sys
sys.path.append("/Users/chixinning/Desktop/kaggle/KaggleMyOwn/ReferenceCode")
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from Help_functions_v2 import sklearn_Pvalue, sklearn_adjR2, RMSE
import numpy as np


In [2]:
from scipy.stats import skew 
from datetime import datetime
from scipy.stats import skew  # for some statistics
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax
from sklearn.linear_model import ElasticNetCV, LassoCV, RidgeCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error
from mlxtend.regressor import StackingCVRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
import matplotlib.pyplot as plt
import scipy.stats as stats
import sklearn.linear_model as linear_model
import seaborn as sns
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [3]:
train_data=pd.read_csv("/Users/chixinning/Desktop/kaggle/KaggleMyOwn/Mycode/raw_data/train.csv")
test=pd.read_csv("test.csv")
print("train_data.shape: ",train_data.shape)
print("test: ",test.shape)

train_data.shape:  (1460, 81)
test:  (1459, 80)


In [4]:
# saving outcome in Sale_Price 有无salePrice的先把sale_Price存起来得了 存的是np.log1p
Sale_Price=np.log1p(train_data.iloc[:,80]
)
Sale_Price.shape


(1460,)

In [5]:
# 我看很多公开的都是结合这俩 很神奇
# data= pd.concat([train,test], keys=['x', 'y'])#here X is training data and Y testing data
# data=data.drop(["Id"],axis=1)
# features = pd.concat([train, test]).reset_index(drop=True)
# features= pd.concat([train,test], keys=['x', 'y'])

In [6]:
# 缺失值处理
quantitative = [f for f in train_data.columns if train_data.dtypes[f] != 'object']
qualitative = [f for f in train_data.columns if train_data.dtypes[f] == 'object']
train=train_data

In [7]:
def encode(frame, feature):
    ordering = pd.DataFrame()
    ordering['val'] = frame[feature].unique()
    ordering.index = ordering.val
    ordering['spmean'] = frame[[feature, 'SalePrice']].groupby(feature).mean()['SalePrice']
    ordering = ordering.sort_values('spmean')
    ordering['ordering'] = range(1, ordering.shape[0]+1)
    ordering = ordering['ordering'].to_dict()
    
    for cat, o in ordering.items():
        frame.loc[frame[feature] == cat, feature+'_E'] = o
    
qual_encoded = []
for q in qualitative:  
    encode(train, q)
    qual_encoded.append(q+'_E')
print(qual_encoded)

['MSZoning_E', 'Street_E', 'Alley_E', 'LotShape_E', 'LandContour_E', 'Utilities_E', 'LotConfig_E', 'LandSlope_E', 'Neighborhood_E', 'Condition1_E', 'Condition2_E', 'BldgType_E', 'HouseStyle_E', 'RoofStyle_E', 'RoofMatl_E', 'Exterior1st_E', 'Exterior2nd_E', 'MasVnrType_E', 'ExterQual_E', 'ExterCond_E', 'Foundation_E', 'BsmtQual_E', 'BsmtCond_E', 'BsmtExposure_E', 'BsmtFinType1_E', 'BsmtFinType2_E', 'Heating_E', 'HeatingQC_E', 'CentralAir_E', 'Electrical_E', 'KitchenQual_E', 'Functional_E', 'FireplaceQu_E', 'GarageType_E', 'GarageFinish_E', 'GarageQual_E', 'GarageCond_E', 'PavedDrive_E', 'PoolQC_E', 'Fence_E', 'MiscFeature_E', 'SaleType_E', 'SaleCondition_E']


In [8]:
# 存完就drop
#droping SalePrice column
train=train_data.drop(["SalePrice"],axis=1)
train.shape

(1460, 123)

In [9]:
# 为什么这里要被drop掉
# train_features = train.drop(['SalePrice'], axis=1)
train_features=train
test_features = test
# 把train_features和test_features拼在一起
features = pd.concat([train_features, test_features]).reset_index(drop=True)

In [10]:
train_data=train_data[train_data.GrLivArea < 4500]
train.reset_index(drop=True, inplace=True)
Sale_Price=Sale_Price.reset_index(drop=True)

In [11]:
features['MSSubClass'] = features['MSSubClass'].apply(str)# numerical -> categorical
features['YrSold'] = features['YrSold'].astype(str)# numerical -> categorical
features['MoSold'] = features['MoSold'].astype(str)# numerical -> categorical
features['Functional'] = features['Functional'].fillna('Typ') #Home functionality (Assume typical unless deductions are warranted)
features['Electrical'] = features['Electrical'].fillna("SBrkr") #   SBrkr	Standard Circuit Breakers & Romex
features['KitchenQual'] = features['KitchenQual'].fillna("TA") #Typical/Average
features["PoolQC"] = features["PoolQC"].fillna("None")#没有就是没有泳池
features['Exterior1st'] = features['Exterior1st'].fillna(features['Exterior1st'].mode()[0]) #Exterior covering on house
features['Exterior2nd'] = features['Exterior2nd'].fillna(features['Exterior2nd'].mode()[0])
# Exterior covering on house
features['SaleType'] = features['SaleType'].fillna(features['SaleType'].mode()[0])

In [12]:
#GarageYrBlt是因为没有车库还是缺失？
for col in ('GarageYrBlt', 'GarageArea', 'GarageCars'):
    features[col] = features[col].fillna(0)
for col in ['GarageType', 'GarageFinish', 'GarageQual', 'GarageCond']:
    features[col] = features[col].fillna('None')
for col in ('BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2'):
    features[col] = features[col].fillna('None')
#MSSubClass: Identifies the type of dwelling involved in the sale.	将MSZoning和MSSubClass相关联
features['MSZoning'] = features.groupby('MSSubClass')['MSZoning'].transform(lambda x: x.fillna(x.mode()[0]))

In [13]:
objects = []
for i in features.columns:
    if features[i].dtype == object:
        objects.append(i)
features.update(features[objects].fillna('None'))
#LotFrontage: Linear feet of street connected to property
features['LotFrontage'] = features.groupby('Neighborhood')['LotFrontage'].transform(lambda x: x.fillna(x.median()))

numeric_dtypes = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numerics = []
for i in features.columns:
    if features[i].dtype in numeric_dtypes:
        numerics.append(i)
features.update(features[numerics].fillna(0))

In [14]:
numeric_dtypes = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numerics2 = []
for i in features.columns:
    if features[i].dtype in numeric_dtypes:
        numerics2.append(i)
# skew_features处理
# skew_features的skewness>0.5的就用boxcox变换处理
skew_features = features[numerics2].apply(lambda x: skew(x)).sort_values(ascending=False)

high_skew = skew_features[skew_features > 0.5]
skew_index = high_skew.index
# ？Q
for i in skew_index:
    features[i] = boxcox1p(features[i], boxcox_normmax(features[i] + 1))

/usr/local/lib/python3.8/site-packages/scipy/stats/stats.py:3845: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/usr/local/lib/python3.8/site-packages/scipy/stats/stats.py:3875: PearsonRNearConstantInputWarning: An input array is nearly constant; the computed correlation coefficent may be inaccurate.
  warnings.warn(PearsonRNearConstantInputWarning())


In [15]:
# Combinations of existing features
features = features.drop(['Utilities', 'Street', 'PoolQC',], axis=1)# 为什么这个feature要drop？是要一个个尝试吗

features['YrBltAndRemod']=features['YearBuilt']+features['YearRemodAdd']
features['TotalSF']=features['TotalBsmtSF'] + features['1stFlrSF'] + features['2ndFlrSF']

features['Total_sqr_footage'] = (features['BsmtFinSF1'] + features['BsmtFinSF2'] +
                                 features['1stFlrSF'] + features['2ndFlrSF'])

features['Total_Bathrooms'] = (features['FullBath'] + (0.5 * features['HalfBath']) +features['BsmtFullBath'] + (0.5 * features['BsmtHalfBath']))
features['Total_porch_sf'] = (features['OpenPorchSF'] + features['3SsnPorch'] +
features['EnclosedPorch'] + features['ScreenPorch'] +features['WoodDeckSF'])

In [16]:
features['haspool'] = features['PoolArea'].apply(lambda x: 1 if x > 0 else 0)
features['has2ndfloor'] = features['2ndFlrSF'].apply(lambda x: 1 if x > 0 else 0)
features['hasgarage'] = features['GarageArea'].apply(lambda x: 1 if x > 0 else 0)
features['hasbsmt'] = features['TotalBsmtSF'].apply(lambda x: 1 if x > 0 else 0)
features['hasfireplace'] = features['Fireplaces'].apply(lambda x: 1 if x > 0 else 0)

In [17]:
features.shape

(2919, 130)

In [18]:
final_features = pd.get_dummies(features).reset_index(drop=True)
final_features.shape
# final_features=features

(2919, 378)

In [19]:
scalerX = MinMaxScaler(feature_range=(0, 1))
final_features[final_features.columns] = scalerX.fit_transform(final_features[final_features.columns])

In [20]:
Train_data = final_features.iloc[:len(Sale_Price), :]
Test_data = final_features.iloc[len(Sale_Price):, :]
Train_data.shape, Sale_Price.shape, Test_data.shape

((1460, 378), (1460,), (1459, 378))

In [21]:
Train_data.insert(2,column="SalePrice",value=Sale_Price)

In [22]:
x=Train_data.drop(["SalePrice"],axis=True)
y=Train_data["SalePrice"]

In [23]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=.30,random_state=40)

In [24]:
#3.1 build multi linear regression 
lr=LinearRegression()
lr.fit(x_train, y_train)
y_trainPred=lr.predict(x_train)
#look at r_squared, adjusted r_squared
print("r-squared of in-sample is")
r2_score(y_train, y_trainPred)
adj_r2=sklearn_adjR2(x_train, y_train)
print("adjusted r-squared of in-sample is")
print(adj_r2)
print("RMSE of in-sample is")
print(RMSE(y_trainPred,y_train))
print(sklearn_Pvalue(x_train, y_train))

# 3.2 out of sample
y_testPred=lr.predict(x_test)
#look at  r_squared, adjusted r_squared
print("r-squared of out-of-sample is")
r2_score(y_test, y_testPred)
print("adjusted r-squared of out-of-sample is")
adj_r2=sklearn_adjR2(x_test, y_test)
print("RMSE of out-of-sample is")
print(RMSE(y_testPred, y_test))
print(sklearn_Pvalue(x_test, y_test))

r-squared of in-sample is
adjusted r-squared of in-sample is
0.9266527662505422
RMSE of in-sample is
0.08766261640145505
     Coefficients    t values  Probabilities
0   -4.016046e-02   54.564925  6.452693e-305
1   -3.146104e-02   93.615255   0.000000e+00
2    4.657857e-01  112.761065   0.000000e+00
3    2.825756e-01  130.472410   0.000000e+00
4    2.741864e-01  141.853540   0.000000e+00
..            ...         ...            ...
373  1.470084e+10    1.672545   9.472321e-02
374  2.098820e+10    2.992457   2.833967e-03
375  1.889242e+10    3.713630   2.153656e-04
376  2.308399e+10   67.987915   0.000000e+00
377  2.517978e+10   10.410994   3.414956e-24

[378 rows x 3 columns]
r-squared of out-of-sample is
adjusted r-squared of out-of-sample is
RMSE of out-of-sample is
3616914258.4266915
     Coefficients   t values  Probabilities
0    2.098589e-03  36.921625  1.989694e-136
1    1.064646e-01  60.248901  8.441823e-214
2    6.668810e-01  83.653482  4.635070e-271
3    1.095679e-01  87.6657

In [25]:
from sklearn.ensemble import RandomForestRegressor
rfr=RandomForestRegressor(n_estimators = 50,random_state=40,
                          min_impurity_decrease=0.002,min_weight_fraction_leaf=0.001,min_samples_split=5)
rfr.fit(x_train,y_train)
y_predictrfc = rfr.predict(x_test)


In [26]:
#here we can check our model score
print(rfr.score(x_test,y_test))
rmse = math.sqrt(mean_squared_error(y_test, rfr.predict(x_test)))

print("mear squares error :",rmse)

0.7973309196254452
mear squares error : 0.17003460114811217


In [27]:
from sklearn.tree import DecisionTreeRegressor
dtr=DecisionTreeRegressor(random_state=140,min_samples_split=5,min_impurity_decrease=0.002,min_weight_fraction_leaf=0.001)
dtr.fit(x_train,y_train)

#u can also use GridSearchCV / random Searchcv for hyperperameter tuning
print(dtr.score(x_test,y_test))

0.6898006201739793


In [28]:
rmse = math.sqrt(mean_squared_error(y_test, dtr.predict(x_test)))
print("RMSE:",rmse)

RMSE: 0.21036041609950504


In [29]:
GBoost = GradientBoostingRegressor(n_estimators=5000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)
#RMSE estimated through the partition of the train set
GBoost.fit(x_train, y_train)

print("RMSE: %.4f" % rmse)

RMSE: 0.2104


In [30]:
y_model_prerfc = GBoost.predict(Test_data)
y_model_prerfc=lr.predict(Test_data)
prediction=np.array(y_model_prerfc).tolist()
test.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [31]:
test.insert(1,column="SalePrice",value=prediction)
test.head()

,Id,SalePrice,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,-5.000193e+10,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,3.779882e+10,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,-2.576406e+10,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,2.282209e+10,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,3.663442e+10,120,RL,43.0,5005,Pave,NaN,IR1,HLS,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [32]:
predict_sub=test.drop(test.iloc[:,2:],axis=1)
predict_sub.head()

,Id,SalePrice
0,1461,-5.000193e+10
1,1462,3.779882e+10
2,1463,-2.576406e+10
3,1464,2.282209e+10
4,1465,3.663442e+10


In [33]:
predict_sub.shape

(1459, 2)

In [34]:
predict_sub.to_csv('Home_predictionsLR.csv',index=False)